In [6]:
!rm -rf my_*.exe

# Compilation chain

## My first program

When all your program fit in a single file, the compiler can apply the full compilation chain, which actually has three steps : **preprocessing**, **compilation**, **linking**. There must be a function `int main( int argc, char * argv[] )`, which will be the entry point of the executable.

Some g++ options:
- `-std=c++17`: apply the C++17 standard.
- `-O2`: reasonably optimize the code.
- `-Wall -Wextra -Werror`: enforce all possible warnings, and make them errors.
- `-pedantic`: forbids any g++ extension which is not ISO/ANSI compliant.
- `-o FILENAME`: name of the produced executable file.

In [7]:
%%file my_program.cpp

#include <iostream>
#include <string>
#include <tuple>
#include <cmath>
#include <cassert>

std::tuple<double,double> polar_coordinates( double x, double y ) {
  double norme = std::sqrt(x*x+y*y) ;
  double angle = std::atan(y/x) ;
  return { norme, angle } ;
}

int main( int argc, char * argv[] ) {
  assert(argc=3) ;
  double x {std::stod(argv[1])} ;
  double y {std::stod(argv[2])} ;
  auto [ n, a ] = polar_coordinates(x,y) ;
  std::cout << n << std::endl ;
  std::cout << a << std::endl ;
  return 0 ; // returns OK to the OS
}

Overwriting my_program.cpp


In [8]:
!g++ -std=c++17 -O2 -Wall -Wextra -Werror -pedantic my_program.cpp -o my_program.exe

In [9]:
!./my_program.exe 1. 1.

1.41421
0.785398


## The preprocessing step

- The compiler delegates this step to the `cpp` preprocessor, which handle the `#` directives, and produc a so-called *translation unit*. Nowadays, this mainly concerns the include of external files.
- The use of C++ macros is highly discouraged, since it has well-known flaws, and most of its features can be better implemented directly in modern C++.
- Yet, the preprocessor is still the last resort solution for some portability issues.

In [10]:
!rm my_*.exe

In [11]:
!g++ -E my_program.cpp

# 1 "my_program.cpp"
# 1 "<built-in>"
# 1 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 1 "<command-line>" 2
# 1 "my_program.cpp"

# 1 "/usr/include/c++/9/iostream" 1 3
# 36 "/usr/include/c++/9/iostream" 3
       
# 37 "/usr/include/c++/9/iostream" 3

# 1 "/usr/include/x86_64-linux-gnu/c++/9/bits/c++config.h" 1 3
# 252 "/usr/include/x86_64-linux-gnu/c++/9/bits/c++config.h" 3

# 252 "/usr/include/x86_64-linux-gnu/c++/9/bits/c++config.h" 3
namespace std
{
  typedef long unsigned int size_t;
  typedef long int ptrdiff_t;


  typedef decltype(nullptr) nullptr_t;

}
# 274 "/usr/include/x86_64-linux-gnu/c++/9/bits/c++config.h" 3
namespace std
{
  inline namespace __cxx11 __attribute__((__abi_tag__ ("cxx11"))) { }
}
namespace __gnu_cxx
{
  inline namespace __cxx11 __attribute__((__abi_tag__ ("cxx11"))) { }
}
# 524 "/usr/include/x86_64-linux-gnu/c++/9/bits/c++config.h" 3
# 1 "/usr/include/x86_64-linux-gnu/c++/9/bits/os_defines.h" 1 3
# 39 "/usr/include/x86_64-linux-gnu/c++/9/bits/o

## The compilation step

- The compiler transforms the output of preprocessing (translation unit) in machine code (object file).
- The previous include directives have only imported the declaration of standard functions.
- The produced object file still lacks the machine code of the needed standard functions (next step).

In [12]:
!g++ -std=c++17 -c my_program.cpp

In [13]:
!ls

01-variables-and-types.ipynb
02-arrays-vs-vectors.ipynb
03-functions-and-references.ipynb
04-compilation.ipynb
05-units.ipynb
06-make.ipynb
my_polar_converter.cc
my_polar_converter.hh
my_polar_converter.o
my_program.cpp
my_program.o


You can inspect what is inside an object file with the command `nm`:

In [14]:
!nm -C my_program.o

                 U __assert_fail
                 U atan
                 U __cxa_atexit
                 U __cxa_begin_catch
                 U __cxa_end_catch
                 U __cxa_rethrow
                 U __dso_handle
0000000000000000 V DW.ref.__gxx_personality_v0
                 U __errno_location
                 U _GLOBAL_OFFSET_TABLE_
000000000000030e t _GLOBAL__sub_I__Z17polar_coordinatesdd
                 U __gxx_personality_v0
000000000000009a T main
                 U sqrt
                 U __stack_chk_fail
                 U strtod
                 U _Unwind_Resume
0000000000000000 T polar_coordinates(double, double)
00000000000002c1 t __static_initialization_and_destruction_0(int, int)
0000000000000000 W __gnu_cxx::char_traits<char>::eq(char const&, char const&)
0000000000000000 W __gnu_cxx::char_traits<char>::length(char const*)
0000000000000000 W bool __gnu_cxx::__is_null_pointer<char const>(char const*)
0000000000000000 W double __gnu_cxx::__stoa<double, double,

## The linking step

- In this final step the OS linker `ld` will join a set of objects files, and link any undefined symbol in one object file, to the corresponding machine code in another object file or library. `g++` autoatically ask `ld` to include the C++ standard library in this process.
- Actually, the standard libraries are generally not fully integrated, so to avoid their duplication in any executable file. The latter remember its needed shared libraries, and search for them whenever you execute it.
- `ldd` inspect the shared libraries needed for a given executable.

In [15]:
!g++ -std=c++17 -O2 -Wall -Wextra -Werror -pedantic my_program.cpp -o my_program.exe

In [16]:
!ldd my_program.exe

	linux-vdso.so.1 (0x00007fff42546000)
	libstdc++.so.6 => /lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007f06cb349000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f06cb1fa000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007f06cb1df000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f06cafed000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f06cb537000)


# Quizz

?

# Take away

- The compilation chain includes **preprocessing**, **compiling** and **linking**.
- Use preprocessor directives only in last resort.
- Pay attention to compilation warnings.
- Keep in mind your executable file generally depends on some shared libraries.


# Questions ?

© *CNRS 2023*  
*This document was created by David Chamont. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*